In [1]:
import os

In [2]:
%pwd

'/Users/lavish./Desktop/Deep Learning/End-to-End-Chest-cancer-Classification-Using-MLFLOW-DVC/research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'/Users/lavish./Desktop/Deep Learning/End-to-End-Chest-cancer-Classification-Using-MLFLOW-DVC'

In [5]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/Lavishgangwani/End-to-End-Chest-cancer-Classification-Using-MLFLOW-DVC.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="LavishGangwani"
os.environ["MLFLOW_TRACKING_PASSWORD"]="51b52d0fd8289a0b86f5bc57fdb7b66677466789"

In [6]:
import tensorflow as tf

In [8]:
model = tf.keras.models.load_model('artifacts/training/trained_model.h5')

## Update the COnfig.yaml

In [9]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model : Path
    training_data : Path
    all_params : dict
    mlflow_uri : str
    params_image_size : list
    params_batch_size : int

In [10]:
from ChestCancerClassifier.constants import *
from ChestCancerClassifier.utils.common import read_yaml,create_directories,save_json

In [13]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath=CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    def get_evaluation_config(self)-> EvaluationConfig:

        evaluation_config = EvaluationConfig(
            path_of_model='artifacts/training/trained_model.h5',
            training_data='artifacts/data_ingestion/Chest-CT-Scan-data',
            mlflow_uri='https://dagshub.com/Lavishgangwani/End-to-End-Chest-cancer-Classification-Using-MLFLOW-DVC.mlflow',
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return evaluation_config

In [14]:
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [15]:
class Evaluation:
    def __init__(self, config:EvaluationConfig):
        self.config = config

    def train_valid_generator(self):
        """
        Sets up data generators for training and validation. 
        Applies data augmentation to the training data if enabled, 
        while validation data remains unaugmented.
        """
        # Common generator arguments
        datagenerator_kwargs = dict(
            rescale=1./255,          # Rescale images
            validation_split=0.20     # Split for validation (20%)
        )

        # Dataflow options
        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],  # Image size without channels
            batch_size=self.config.params_batch_size,        # Batch size
            interpolation="bilinear"                        # Interpolation method
        )

        # Validation data generator (no augmentation)
        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        # Validation generator setup
        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    
    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self.train_valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)


    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")

In [17]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

Found 68 images belonging to 2 classes.


/opt/anaconda3/envs/chest/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


5/5 ━━━━━━━━━━━━━━━━━━━━ 7s 1s/step - accuracy: 1.0000 - loss: 0.1498


2024/10/25 12:25:00 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.
2024/10/25 12:25:03 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Registered model 'VGG16Model' already exists. Creating a new version of this model...
2024/10/25 12:27:00 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: VGG16Model, version 2
Created version '2' of model 'VGG16Model'.
2024/10/25 12:27:01 INFO mlflow.tracking._tracking_service.client: 🏃 View run rumbling-conch-911 at: https://dagshub.com/Lavishgangwani/End-to-End-Chest-cancer-Classification-Using-MLFLOW-DVC.mlflow/#/experiments/0/runs/44bd7814f9fc489ba956864814fc5d7f.
2024/10/25 12:27:01 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/Lavishgangwani/End-to-End-Chest-c